In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, json
from astropy import constants as const
from astropy import units as u

# increase label text size to 14
font = {'size': 14}
plt.rc('font', **font)

### Load data

In [ ]:
data = pd.read_csv(DATA_DIR + "/neutron_att_coeff.csv", sep=',')
print(data)

## Neutron attenuation